In [22]:
from __future__ import print_function

import json
import logging
import numpy as np
import shutil
import os
import pandas as pd
import requests
import itertools
from collections import OrderedDict
from datetime import datetime
from pathlib import Path
from tqdm import tqdm

import skimage.transform as skt
import skimage

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

# Interactive/Widgets
import ipywidgets as widgets
from IPython.core.display import display, HTML
from ipywidgets import interact
from ipywidgets import interact, interactive, fixed, interact_manual
import qgrid
import wandb

# brtdevkit
import brtdevkit
from brtdevkit.core.db import DBConnector
from brtdevkit.core.db.db_filters import *  # We need this for pre-defined filters, e.g., ProjectFilter, DatetimeFilter
import brtdevkit.util.s3 as brt_s3

# dl-core
import brtdl.metrics
from brtdl.metrics.evaluate_metrics import evaluate_metrics
from brtdl import inference, default_arguments
from brtdl.data import canonical_types, SampleKeys
from brtdl.data.dataset import PathDataset, PredictionDataset
from brtdl.data.loaders import im_reader, npy_reader
from brtdl import inference, default_arguments
from brtdl.transforms import SegmentationTransform
from brtdl.visual import colorize_segmentation

%matplotlib inline
logging.basicConfig(level=logging.INFO)

In [70]:
import time
def get_shasta_data(filters={}, start=None, end=None, limit=None):
    """
    Query Shasta metadata matching filters.
    """
    start_time = time.time()
    connector = DBConnector()
    img_filters = {'project_name': 'shasta', **filters}
    if start is not None or end is not None:
        img_filters = [img_filters, DatetimeFilter(key="collected_on", start=start, end=end)]
    df = connector.get_documents_df('image', img_filters, limit=limit)
    elapsed_time = time.time() - start_time
    return df, elapsed_time

# Set date filters to return data collected between 03/01/2020 and 09/01/2020
start = datetime(2020,12,1)
end = datetime(2021, 1,1)

# Lists of DCMs and Machines
dcms = ['DCM-MANATEE', 'DCM-WALRUS', 'DCM-SEAL', 'DCM-OTTER', 'DCM-PORPOISE', 'DCM-DOLPHIN']
machines = ["SHASTA-FB-BRADLEY","SHASTA-FB-PALADIN", "BLACKBIRD", 'ATM-DUCKDUCK', 'ATM-GOOSE']
valid_isp_versions = ['07080203','07090000','07090100']

# Select filters
filters = { "artifacts.kind": "nrg",  
           'robot_name':{"$in":['DCM-OTTER' , 'DCM-PORPOISE']}
          }

full_df, elapsed_time = get_shasta_data(filters=filters, start = start, end = end)
full_df['date_collected'] = pd.to_datetime(full_df['collected_on']).dt.date

print(f"Queried {len(full_df)} images in {elapsed_time:.2f} s.")

/home/williamroberts/code/brtdevkit/venv/lib/python3.6/site-packages/brtdevkit/core/db/db_connector.py:314: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat(chunks) if len(chunks) > 0 else pd.DataFrame()


Queried 18870 images in 27.48 s.


In [64]:
# Helper Functions

def download_images(df, artifact_kind, outputdir):
    """
    Downloads images from S3 given a dataframe.
    :param df: dataframe, expected to be created by a query to brtdevkit
    :param artifact_kind: string
    :param outputdir: output directory to store the downloaded images
    """
    s3_client = brt_s3.S3()
    for ix, row in tqdm(df.iterrows(), total=df.shape[0]):
        art = [x for x in row.artifacts if x['kind'] == artifact_kind]
        if art:
            art = art[0]
            fname = os.path.join(outputdir, os.path.basename(art['s3_key']))
            s3_client.download_file(art['s3_bucket'], art['s3_key'], fname)
            
def download_from_url(url, fname):
    """
    Helper function to download data given an URL
    :param url: string, url of the file to be downloaded
    :param fname: filename of the downloaded file
    """
    r = requests.get(url, allow_redirects=False)
    with open(fname, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)     


### Set Up Variables For Inference

In [89]:
def get_id_from_s3(df, key):
    for ix, row in df.iterrows():
        art = [x for x in row.artifacts if x['kind'] == 'nrg']
        if art:
            art = art[0]
            #print(art['s3_key'][19:])
            if art['s3_key'][19:] == key:
                print(row['_id']) 

get_id_from_s3(full_df, '4f6698db6f674adea7c8d170dbd2d59f_nrg.png')

5fcf35a94f3275985d2159cc


In [48]:
channel_order = 'HWC'
batch_size = 1
operating_field_name = '27'
#farm = 'ciarama itaguassu'
grower = 'jotabasso'

crop_name = 'SOYBEANS'
limit = 200
urls = ['https://artifactory.bluerivertech.com/artifactory/dev-shasta-models/jit/20200528_2_soybeans/20200528_2_soybeans.jit',
        'https://artifactory.bluerivertech.com:443/artifactory/dev-shasta-models/jit/20200923_1_soybeans_5/20200923_1_soybeans_5.jit',
        'https://artifactory.bluerivertech.com:443/artifactory/dev-shasta-models/jit/20201104_1_soybeans_5/20201104_1_soybeans_5.jit',
       'https://artifactory.bluerivertech.com:443/artifactory/dev-shasta-models/jit/20201101_1_soybeans_4/20201101_1_soybeans_4.jit']

model_url = urls[0]
out_name = f'{grower.replace(" ", "")}_{operating_field_name.replace(" ", "")}_{crop_name.replace(" ", "")}'
out_dir = Path('') / Path(out_name)
SHOW_IMAGES = False

connector = DBConnector()
  
"""
Get the initial dataset from brtdevkit
"""
image_initial = connector.get_documents_df('image', [ProjectFilter('shasta'),
                                                ValueFilter('crop_name', crop_name),
                                                #ValueFilter('farm', farm),
                                                ValueFilter('grower', grower),
                                                ValueFilter('operating_field_name', operating_field_name)], limit = limit)
print(len(image_initial))
image_initial.head()

200


,_id,is_deleted,collected_on,gnss,annotations,artifacts,has_human_annotation,uuid,artifacts_content_hash,angle_to_row,...,row_spacing,soil_color,soil_moisture,tillage_practice,type,weed_pressure,weeds,created_at,updated_at,collection_name
_id,,,,,,,,,,,,,,,,,,,,,
5fd9c0e76fdb3db1807ed13c,5fd9c0e76fdb3db1807ed13c,False,2020-12-15 18:07:49.891,"{'type': 'Point', 'coordinates': [-55.63839721...","[{'_id': 5fd9c0e76fdb3db1807ed13f, '_cls': 'Pi...","[{'_id': 5fd9c0e76fdb3db1807ed13b, 'kind': 'ra...",False,88871b4d-1c82-4ba0-8060-c3776c021745,0d0800728f731c7f985565568473223a;82d5bc3ed766f...,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,nrg,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:15,NaT,image
5fd9c0e76fdb3db1807ed144,5fd9c0e76fdb3db1807ed144,False,2020-12-15 18:07:49.907,"{'type': 'Point', 'coordinates': [-55.63839721...",[],"[{'_id': 5fd9c0e76fdb3db1807ed143, 'kind': 'rg...",False,09240523-8725-4630-8ff3-8f3645de1226,9fcbf8d7e6fba3a99aab4bd35358c352,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,rgb,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:15,NaT,image
5fd9c0e76fdb3db1807ed146,5fd9c0e76fdb3db1807ed146,False,2020-12-15 18:07:50.891,"{'type': 'Point', 'coordinates': [-55.63841247...","[{'_id': 5fd9c0e76fdb3db1807ed149, '_cls': 'Pi...","[{'_id': 5fd9c0e76fdb3db1807ed145, 'kind': 'fa...",False,82a54e91-ea1d-4728-9716-b05eba535f09,0160e47393dff1d93114153bddde22ca;64d70c8acf29f...,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,nrg,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:15,NaT,image
5fd9c0e86fdb3db1807ed14e,5fd9c0e86fdb3db1807ed14e,False,2020-12-15 18:07:50.907,"{'type': 'Point', 'coordinates': [-55.63841247...",[],"[{'_id': 5fd9c0e86fdb3db1807ed14d, 'kind': 'rg...",False,1a0ff4db-fe3e-4fe6-aa42-05d8d49e9015,7fed6fc43628ecf60aaec6524eeb2614,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,rgb,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:16,NaT,image
5fd9c0ea6fdb3db1807ed1b4,5fd9c0ea6fdb3db1807ed1b4,False,2020-12-15 18:07:51.891,"{'type': 'Point', 'coordinates': [-55.63842773...","[{'_id': 5fd9c0ea6fdb3db1807ed1b8, '_cls': 'Pi...","[{'_id': 5fd9c0ea6fdb3db1807ed1b3, 'kind': 'fa...",False,e18e6bcf-a2ba-4dbf-b1ae-529fcd7f9038,07892ef2e9f9ce7ab3bd5596578327d8;1b8d27d615675...,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,nrg,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:18,NaT,image


### Set Up Directories For Images

In [49]:
# Setup directories
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
out_dir.mkdir(parents=False, exist_ok=True)

model_dir = out_dir / Path('ShastaModel')
model_dir.mkdir(parents=False, exist_ok=True)

img_dir = out_dir / Path('nrgimages')
img_dir.mkdir(parents=False, exist_ok=True)

result_dir = out_dir / Path('results')
result_dir.mkdir(parents=False, exist_ok=True)

### Download Images

In [50]:
download_images(image_initial, 'nrg', img_dir)

100%|██████████| 200/200 [01:34<00:00,  2.11it/s]


### Load The Model

In [28]:
import os
model_url ='https://artifactory.bluerivertech.com:443/artifactory/dev-shasta-models/jit/20210119_1_soybeans_4/20210119_1_soybeans_4.jit'
# 'https://artifactory.bluerivertech.com:443/artifactory/dev-shasta-models/jit/20210119_1_soybeans_4/20210119_1_soybeans_4.jit'
model_name = os.path.basename(model_url)
model_path = model_dir / Path(model_name)
download_from_url(model_url, model_path)

metadata_name = 'metadata.json'
metadata_url = os.path.join(os.path.dirname(model_url), 'metadata.json')
metadata_path = model_dir / Path(metadata_name)
download_from_url(metadata_url, metadata_path)

with open(metadata_path, 'r') as f:
    model_metadata = json.load(f)
    
print(model_metadata)

{'rank': 1, 'creation_date': '20210119', 'model_type': 'semantic_segmentation', 'model_architecture': 'TruncatedPyramid4x', 'crop_type': ['soybeans'], 'input_order': 'NCHW', 'input_shape': [4, 3, 540, 960], 'input_data_format': 'NRG', 'input_data_type': 'float32', 'input_data_range': 'rangenorm', 'output_shape': [135, 240], 'output_data_format': {'0': 'ground', '1': 'weed', '2': 'crop'}, 'config_parameters': {'train_info_link': '6006973dec8e521de159960d'}, 'image_quality': {}, 'release_version': ['20210119', 1, 'soybeans', 4]}


In [29]:
print(model_path)

gvr_th06_SOYBEANS/ShastaModel/20210119_1_soybeans_4.jit


### Run Inference and Generate Images

In [51]:
resize_shape = (540, 960)#(model_metadata['input_shape'][2], model_metadata['input_shape'][3])

with inference.managed_indir(default_arguments.PathTypes.abspath(img_dir)) as data_dir:
    suffix = inference.infer_suffix(data_dir, '')
    dset = inference.dset_dispatcher[suffix](data_dir, suffix, resize_shape, channel_order)
#model_path = 'gvr_th06_SOYBEANS/ShastaModel/20210119_1_soybeans_4.jit'
# This was a separate cell in the original notebook
pred_gen = inference.predict_jit(dset, default_arguments.PathTypes.abspath(model_path), batch_size, device='cpu')

# This was a separate cell in the original notebook
# tqdm(pred_gen, total=len(dset))
for img_id, f_hat in pred_gen:
    y_hat = np.argmax(f_hat, axis=0).astype(np.ubyte)
    img = mpimg.imread(os.path.join(img_dir, img_id + suffix)) * 255.0
    img = img.astype(np.uint8)
    h, w, c = img.shape
    resized_yhat = skimage.transform.resize(y_hat, (h, w), order=0,
                                            mode="constant", anti_aliasing=False, 
                                            preserve_range=True)
    resized_yhat = resized_yhat.astype(np.uint8)
    colorized_img = colorize_segmentation(img, resized_yhat)
    out_path = os.path.join(result_dir, img_id + ".jpg")
    skimage.io.imsave(out_path, colorized_img)

# This was a separate cell in the original notebook
# make a little json descriptor file for the web viewer
desc = {'model': os.path.basename(model_path),
        'grower': grower,
        'operating_field_name': operating_field_name,
        'crop_name': crop_name,
        'results_dir': os.path.basename(result_dir)}

with open(str(out_dir / Path('description.json')), 'w', encoding='utf-8') as f:
    json.dump(desc, f, ensure_ascii=True, indent=4)
    
if SHOW_IMAGES:
    img_list = list(result_dir.glob('*.jpg'))
    #img_list = img_list[0:40]
    imgs = []
    for fname in img_list:
        img = plt.imread(fname)
        imgs.append(img)

        from mpl_toolkits.axes_grid1 import ImageGrid
    fig = plt.figure(figsize=(400., 400.))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(len(img_list) // 2, 2),  # creates 2x2 grid of axes
                     axes_pad=0.2,  # pad between axes in inch.
                     )

    for ax, im in zip(grid, imgs):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)

    plt.show()
print(f'Done with {grower+operating_field_name}')

/home/williamroberts/code/brtdevkit/venv/lib/python3.6/site-packages/imgaug/augmenters/meta.py:3091: DeprecationWarning: The parameter `deterministic` is deprecated in `imgaug.augmenters.meta.Augmenter`. Use `.to_deterministic()` to switch into deterministic mode.
  random_state=random_state, deterministic=deterministic)


Done with jotabasso27


In [53]:
image_initial

,_id,is_deleted,collected_on,gnss,annotations,artifacts,has_human_annotation,uuid,artifacts_content_hash,angle_to_row,...,row_spacing,soil_color,soil_moisture,tillage_practice,type,weed_pressure,weeds,created_at,updated_at,collection_name
_id,,,,,,,,,,,,,,,,,,,,,
5fd9c0e76fdb3db1807ed13c,5fd9c0e76fdb3db1807ed13c,False,2020-12-15 18:07:49.891,"{'type': 'Point', 'coordinates': [-55.63839721...","[{'_id': 5fd9c0e76fdb3db1807ed13f, '_cls': 'Pi...","[{'_id': 5fd9c0e76fdb3db1807ed13b, 'kind': 'ra...",False,88871b4d-1c82-4ba0-8060-c3776c021745,0d0800728f731c7f985565568473223a;82d5bc3ed766f...,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,nrg,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:15,NaT,image
5fd9c0e76fdb3db1807ed144,5fd9c0e76fdb3db1807ed144,False,2020-12-15 18:07:49.907,"{'type': 'Point', 'coordinates': [-55.63839721...",[],"[{'_id': 5fd9c0e76fdb3db1807ed143, 'kind': 'rg...",False,09240523-8725-4630-8ff3-8f3645de1226,9fcbf8d7e6fba3a99aab4bd35358c352,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,rgb,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:15,NaT,image
5fd9c0e76fdb3db1807ed146,5fd9c0e76fdb3db1807ed146,False,2020-12-15 18:07:50.891,"{'type': 'Point', 'coordinates': [-55.63841247...","[{'_id': 5fd9c0e76fdb3db1807ed149, '_cls': 'Pi...","[{'_id': 5fd9c0e76fdb3db1807ed145, 'kind': 'fa...",False,82a54e91-ea1d-4728-9716-b05eba535f09,0160e47393dff1d93114153bddde22ca;64d70c8acf29f...,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,nrg,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:15,NaT,image
5fd9c0e86fdb3db1807ed14e,5fd9c0e86fdb3db1807ed14e,False,2020-12-15 18:07:50.907,"{'type': 'Point', 'coordinates': [-55.63841247...",[],"[{'_id': 5fd9c0e86fdb3db1807ed14d, 'kind': 'rg...",False,1a0ff4db-fe3e-4fe6-aa42-05d8d49e9015,7fed6fc43628ecf60aaec6524eeb2614,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,rgb,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:16,NaT,image
5fd9c0ea6fdb3db1807ed1b4,5fd9c0ea6fdb3db1807ed1b4,False,2020-12-15 18:07:51.891,"{'type': 'Point', 'coordinates': [-55.63842773...","[{'_id': 5fd9c0ea6fdb3db1807ed1b8, '_cls': 'Pi...","[{'_id': 5fd9c0ea6fdb3db1807ed1b3, 'kind': 'fa...",False,e18e6bcf-a2ba-4dbf-b1ae-529fcd7f9038,07892ef2e9f9ce7ab3bd5596578327d8;1b8d27d615675...,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,nrg,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:10:18,NaT,image
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5fd9c1b78b151c63d300514b,5fd9c1b78b151c63d300514b,False,2020-12-15 18:10:12.907,"{'type': 'Point', 'coordinates': [-55.63899230...",[],"[{'_id': 5fd9c1b78b151c63d300514a, 'kind': 'nr...",False,a06bfb53-a44c-4a25-9e45-58621bac6317,2aa79e8a9514d156b1bd6a02fd679b66,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,nrg,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:13:43,NaT,image
5fd9c1b78b151c63d300514d,5fd9c1b78b151c63d300514d,False,2020-12-15 18:10:12.907,"{'type': 'Point', 'coordinates': [-55.63899230...",[],"[{'_id': 5fd9c1b78b151c63d300514c, 'kind': 'rg...",False,01bed225-4ef9-4337-ac50-e9fc345532d5,f8f31752f43d6447433e48cbc067245f,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,rgb,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:13:43,NaT,image
5fd9c1b78b151c63d300514f,5fd9c1b78b151c63d300514f,False,2020-12-15 18:10:13.891,"{'type': 'Point', 'coordinates': [-55.63898849...","[{'_id': 5fd9c1b78b151c63d3005153, '_cls': 'Pi...","[{'_id': 5fd9c1b78b151c63d300514e, 'kind': 'fa...",False,f30b5998-f482-4057-97ad-566dc08e8dfb,2d8494b5fa67372f259fe984c70b609d;2e4abf7e320b5...,PARALLEL,...,CENTIMETERS_45,OTHER,UNKNOWN,OTHER,nrg,MEDIUM,"[BUVA, CAPIM_COLCHAO, GUANXUMA]",2020-12-16 08:13:43,NaT,image
